In [1]:
import numpy as np

from xgboost import XGBRegressor, DMatrix
from xgboost.callback import TrainingCallback
from xgboost.core import Booster

from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
n = 10000
features = [
    np.random.normal(0, 1, n) for _ in range(100)
]

error = np.random.normal(0, 1, n)

y = sum(features) + error
X = np.column_stack(features)

In [3]:
X_train = X[:9000, :]
y_train = y[:9000]

X_test = X[9000:, :]
y_test = y[9000:]

## Jointly train two XGBoost models

First to it on the boosters directly

In [4]:
xgb = XGBRegressor()

In [5]:
params = xgb.get_xgb_params()
params

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'eval_metric': None,
 'gamma': None,
 'grow_policy': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [6]:
X_train.shape

(9000, 100)

In [7]:
X_train1 = X_train[:, :50]
X_train2 = X_train[:, 50:]
X_train1.shape, X_train2.shape

((9000, 50), (9000, 50))

In [8]:
X_test1 = X_test[:, :50]
X_test2 = X_test[:, 50:]
X_test1.shape, X_test2.shape

((1000, 50), (1000, 50))

Set up the XGBoost data

In [9]:
preds1 = np.zeros(X_train.shape[0])
preds2 = np.zeros(X_train.shape[0])

dtrain1 = DMatrix(np.column_stack([X_train1, preds2]), label=y_train)
dtrain2 = DMatrix(np.column_stack([X_train2, preds1]), label=y_train)

preds1_test = np.zeros(X_test.shape[0])
preds2_test = np.zeros(X_test.shape[0])

dtest1 = DMatrix(np.column_stack([X_test1, preds2_test]))
dtest2 = DMatrix(np.column_stack([X_test2, preds1_test]))

In [10]:
bst1 = Booster(params, [dtrain1])
bst2 = Booster(params, [dtrain2])

In [11]:
def mask_values(array, prop=0.3):
    # randomly drop some indices
    out = array.copy()
    rand = np.random.choice(out.shape[0], (int(out.shape[0] * prop)), replace=False)
    out[rand] = np.nan
    return out

In [12]:
start_iteration = 0
num_boost_round = xgb.get_num_boosting_rounds()
print(num_boost_round)

preds1 = np.zeros(X_train.shape[0])
preds2 = np.zeros(X_train.shape[0])

bst1.update(dtrain1, iteration=0, fobj=None)
bst2.update(dtrain2, iteration=0, fobj=None)

for i in range(start_iteration+1, num_boost_round):
    # randomly drop some values
    preds2_masked = mask_values(preds2)
    array1 = np.column_stack([X_train1, preds2_masked])
    dtrain1 = DMatrix(array1, label=y_train)
    bst1.update(dtrain1, iteration=i, fobj=None)
    preds1 = bst1.predict(dtrain1)

    preds1_masked = mask_values(preds1)
    array2 = np.column_stack([X_train2, preds1_masked])
    dtrain2 = DMatrix(array2, label=y_train)
    bst2.update(dtrain2, iteration=i, fobj=None)
    preds2 = bst2.predict(dtrain2)

100


## Model Performance

In [13]:
# train separate models
xgb1 = XGBRegressor()
xgb2 = XGBRegressor()

In [14]:
xgb1.fit(X_train1, y_train)
xgb2.fit(X_train2, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
pred1_individual = xgb1.predict(X_test1)
pred2_individual = xgb2.predict(X_test2)

In [16]:
root_mean_squared_error(pred1_individual, y_test), root_mean_squared_error(pred2_individual, y_test)

(np.float64(8.336413831232552), np.float64(8.355896739085315))

In [17]:
dtest1_pred = DMatrix(np.column_stack([
    X_test1, np.full(X_test1.shape[0], np.nan)
]))

dtest2_pred = DMatrix(np.column_stack([
    X_test2, np.full(X_test2.shape[0], np.nan)
]))

In [18]:
pred1_joint = bst1.predict(dtest1_pred)
pred2_joint = bst2.predict(dtest2_pred)

In [19]:
root_mean_squared_error(pred1_joint, y_test), root_mean_squared_error(pred2_joint, y_test)

(np.float64(8.555848545377652), np.float64(8.370973017764538))

In [20]:
# now, fill with the other model's predictions
dtest1_pred = DMatrix(np.column_stack([
    X_test1, pred2_joint
]))
dtest2_pred = DMatrix(np.column_stack([
    X_test2, pred1_joint
]))

In [21]:
pred1_joint = bst1.predict(dtest1_pred)
pred2_joint = bst2.predict(dtest2_pred)

In [22]:
root_mean_squared_error(pred1_joint, y_test), root_mean_squared_error(pred2_joint, y_test)

(np.float64(6.8802572771662245), np.float64(7.051215790266453))

Does doing another round help? No - it seems worse than before

In [23]:
# now, fill with the other model's predictions
dtest1_pred = DMatrix(np.column_stack([
    X_test1, pred2_joint
]))
dtest2_pred = DMatrix(np.column_stack([
    X_test2, pred1_joint
]))

In [24]:
pred1_joint2 = bst1.predict(dtest1_pred)
pred2_joint2 = bst2.predict(dtest2_pred)

In [25]:
root_mean_squared_error(pred1_joint2, y_test), root_mean_squared_error(pred2_joint2, y_test)

(np.float64(7.508733546099677), np.float64(7.204603321415622))

## Joint Training, But Only One Round

In [26]:
xgb_joint_1 = XGBRegressor()
xgb_joint_1.fit(X_train1, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
xgb_joint_2 = XGBRegressor()
xgb_joint_2.fit(X_train2, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [28]:
xgb_pred_1 = xgb_joint_1.predict(X_train1)
xgb_pred_2 = xgb_joint_2.predict(X_train2)

In [29]:
xgb_joint_1 = XGBRegressor()
xgb_joint_1.fit(np.column_stack([X_train1, xgb_pred_2]), y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [30]:
pred_joint_separate2 = xgb_joint_2.predict(X_test2)

In [31]:
pred_joint_separate1 = xgb_joint_1.predict(np.column_stack([X_test1, pred_joint_separate2]))

In [32]:
rmse_joint_separate1 = root_mean_squared_error(pred_joint_separate1, y_test)
rmse_joint_separate2 = root_mean_squared_error(pred_joint_separate2, y_test)

rmse_joint_separate1, rmse_joint_separate2

(np.float64(7.540299495846418), np.float64(8.355896739085315))

In comparison, the true joint training performs better.

In [33]:
root_mean_squared_error(pred1_joint, y_test), root_mean_squared_error(pred2_joint, y_test)

(np.float64(6.8802572771662245), np.float64(7.051215790266453))

## Model Trained on Full Dataset

For comparison, the model trained on the full dataset still does slightly better.

In [34]:
xgb_combined = XGBRegressor(n_estimators=200)

In [35]:
xgb_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [36]:
pred_combined = xgb_combined.predict(X_test)

In [37]:
rmse_combined = root_mean_squared_error(pred_combined, y_test)
rmse_combined

np.float64(6.838734862199035)

But performance of the average of the two joint models is better? Maybe similar to bagging

In [38]:
pred_average = (pred1_joint + pred2_joint) / 2
rmse_average = root_mean_squared_error(pred_average, y_test)
rmse_average

np.float64(6.637759522448021)

In [39]:
(rmse_combined - rmse_average) / rmse_combined

np.float64(0.02938779522831066)

This actually does do better than averaging the individual predictions.

In [40]:
pred_average_individual = (pred1_individual + pred2_individual) / 2
root_mean_squared_error(pred_average_individual, y_test)

np.float64(6.992794948210651)